In [65]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [73]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [74]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [75]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [76]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [77]:
def save_report(file_path, df):
    print(df)
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)

In [78]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response
    except Exception as e:
        print(e)
        

In [79]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(5):
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

Prompt==> {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-2d89737e3add4c7aa9722110320461cf","object":"text_completion","created":435034,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef reverse_string(s):\n    return s[::-1]\n\nprint(reverse_string(\"Hello World\"))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":68,"completion_tokens":32}}
Execution time: 1.913007 seconds 


Prompt==> {"payload":"Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome."} 
 {"id":"cmpl-5eef1eec8f3d4887a2c1d1b001ce8c0e","object":"text_completion","created":435034,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices"

Prompt==> {"payload":"Problem: Write a function in java to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s."} 
 {"id":"cmpl-67e08b4ad8c74fac8d43031f493ea362","object":"text_completion","created":435042,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":42,"completion_tokens":1}}
Execution time: 0.105896 seconds 


Prompt==> {"payload":"Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-5caf767903104c70849a79955d34609e","object":"text_completion","created":435042,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprob

Prompt==> {"payload":"Problem: Write a function in c++ to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."} 
 {"id":"cmpl-5d352ccb59944117b4e35f08cb5b604e","object":"text_completion","created":435043,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nHere is a simple C++ function to check if a given number is prime:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nbool isPrime(int num) {\n    if (num <= 1) {\n        return false;\n    }\n    for (int i = 2; i * i <= num; i++) {\n        if (num % i == 0) {\n            return false;\n        }\n    }\n    return true;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a number: \";\n    cin >> num;\n    if (isPrime(num)) {\n        cout << num << \" is a prime number.\" << endl;\n    } else {\n        cout << num << \" is not a prime number.\" << endl;\n    }\n  

Prompt==> {"payload":"Problem: Write a function in c++ to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-021b66bbf70c4074afbedaa0b5735e4f","object":"text_completion","created":435052,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n#include <iostream>\n#include <vector>\n\nint binarySearch(std::vector<int>& arr, int target) {\n    int left = 0;\n    int right = arr.size() - 1;\n\n    while (left <= right) {\n        int mid = left + (right - left) / 2;\n\n        if (arr[mid] == target)\n            return mid;\n\n        if (arr[mid] < target)\n            left = mid + 1;\n        else\n            right = mid - 1;\n    }\n\n    return -1;\n}\n\nint main() {\n    std::vector<int> arr = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};\n    int target = 5;\n\n    int","logprobs":null,"fin

Prompt==> {"payload":"Problem: Write a function in Golang to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-a58d720f249f4b8c937bd5853bbcd847","object":"text_completion","created":435068,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":38,"total_tokens":39,"completion_tokens":1}}
Execution time: 0.108500 seconds 


Prompt==> {"payload":"Problem: Write a function in Golang to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."} 
 {"id":"cmpl-277d37497dbd45899c58d8bb6779bf46","object":"text_completion","created":435068,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt

Prompt==> {"payload":"Problem: Write a function in CSS that dynamically creates an HTML element and appends it to the document.Input: String, element type (e.g., 'div', 'button').Output: None (update the HTML document)."} 
 {"id":"cmpl-ba95fd375fbb4852bbd7e86d0d43d850","object":"text_completion","created":435069,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":55,"total_tokens":56,"completion_tokens":1}}
Execution time: 0.122942 seconds 


Prompt==> {"payload":"Problem: Implement a function in CSS to validate an email address.Input: String, email address.Output: Boolean, indicating whether the email address is valid."} 
 {"id":"cmpl-c942b3261b75429cbc37db8aa1dc0e77","object":"text_completion","created":435069,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{

Prompt==> {"payload":"Problem: Create an animated dropdown menu in angular that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu)."} 
 {"id":"cmpl-93cf3e3672564814bcc103f3cb534d29","object":"text_completion","created":435070,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}
Execution time: 0.122311 seconds 


Prompt==> {"payload":"Problem: Implement form validation in angular to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid."} 
 {"id":"cmpl-ec25ac892e8944e9beb09a88dcaa43b6","object":"text_completion","created":435070,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c

Prompt==> {"payload":"Problem: Create a function in jQuery to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element)."} 
 {"id":"cmpl-b2b500a17f354f9f923cad9e8ee2244b","object":"text_completion","created":435071,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":50,"total_tokens":51,"completion_tokens":1}}
Execution time: 0.108707 seconds 


Prompt==> {"payload":"Problem: Write a function in Ruby to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams."} 
 {"id":"cmpl-3fda720c5e2b400cac87e1a25af51c2c","object":"text_completion","created":435065,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))